<a href="https://colab.research.google.com/github/vishvas-chauhan/NLP-ML/blob/main/TensorFlow_text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import 3 libraries 
#tensorflow is opensource, developed by google brain team, good and work on potential words only and avoid words like 'i', 'am', 'is' etc.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds #sample datasets from google tensorflow. 
import tensorflow_hub as hub # module use for pretrained datasets. Till now I used only for NLP purpose only. It Can save a lot of time. 

In [ ]:
tf.__version__

'2.2.0'

In [ ]:
#import tensorflow_text

In [ ]:
tfds.list_builders() #inbuild dataset list provided by google team, I will use 'imdb review'

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

In [ ]:
splits= ("train[:80]",'train[:10]','train[:10]')
(train_data, validation_data, test_data),metadeta= tfds.load(name='imdb_reviews',split=splits,with_info=True,as_supervised=True) 
#we will load and split the data

In [ ]:
train_example_batch, train_label_batch = next(iter(train_data.batch(2))) 
#split numpy_batched into smaple and classified way (0,1 only), lets check with first 2 comments 


In [ ]:
train_example_batch
#both are negative or toxic comments..'This was an absolutely terrible movie'.. Agree?

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [ ]:
train_label_batch
#negative comment means 0 and postive you know ?

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0])>

In [ ]:
#It was defined in given datasets. Now lets make and train a model

In [ ]:
#pretrained_model= you go for https://tfhub.dev and you will find NLP pretrained hub (transfering models from TensorFlow hub )

In [ ]:
#copy paste model link according to your requirement, below one is trained on 130GB text data so It was not a bad choice. Agree?

pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

#making a layer and then we will add this layer in our model for predictions. Its not that tricky right?
#it converts pretrained model to vector or say from WORDS TO NUMBERS

hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True, name='bert_embedding')

In [ ]:
#apply word2vector hub_layer ON over first comments. if its working. 
hub_layer(train_example_batch[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

In [ ]:
#we dont need to understand these numbers. As i said it randomly selected 20 potential words only. 

In [ ]:
train_example_batch[:1] #there are lots of words though. It saved our 50% of the time

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
      dtype=object)>

In [ ]:
#Now make a model with few neurons network 20, 16 and make sure 1 in the end to avoid confusion. So that it shows same prediction each time

In [ ]:
model = tf.keras.Sequential() #starts with sequestial neurons 
model.add(hub_layer) #adding pretrained layer. Must check how it was trained by google, just copy paste URL. I already did. 
model.add(tf.keras.layers.Dense(16,activation="relu")) # relu method 
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))# then can switch the method. I will try too. if it imporves the prediction lets see. You also try
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_embedding (KerasLayer)  (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#our all neurons are trainable. (20, 16, 1) are neurons and param calculation = input values+ neurons + bias neurons ['read articles about it']

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) 

In [ ]:
#Compile defines the loss function, the optimizer and the metrics. 
#That's all. It has nothing to do with the weights and you can compile a model as many times as you want 
#You need a compiled model to train (because training uses the loss function and the optimizer)

In [ ]:
 #model fitting by shuffles 20000 data in batches of 512 in 20 epochs. 
 #epochs - number of times all training vectors are used as per weight 
 #batches are just training samples 
 model.fit(train_data.shuffle(10000).batch(512),epochs=20, validation_data=validation_data.batch(512),verbose=1)

NameError: ignored

In [ ]:
model.predict([['this is worst movie. Acting was bad']])

array([[0.393671]], dtype=float32)

In [ ]:
model.predict([['I loved movie, favourite']])

array([[0.7961829]], dtype=float32)

In [ ]:
#model is working fine now. #happylearning #SingSongs #DontRushSlowTouch

In [ ]:
#open for team, welcome, I will try to change the optimiser mannually and hub layer. 

Available optimizers
1. SGD
2. RMSprop
3. Adam
4. Adadelta
5. Adagrad
6. Adamax
7. Nadam
8. Ftrl

In [ ]:
optimizer_M = tf.keras.optimizers.Adam()

# Iterate over the batches of a dataset.
for x, y in train_data:
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        # Forward pass.
        logits = model(model = tf.keras.Sequential() #starts with sequestial neurons 
                       model.add(hub_layer) #adding pretrained layer. Must check how it was trained by google, just copy paste URL. I already did. 
                       model.add(tf.keras.layers.Dense(16,activation="relu")) # relu method 
                       model.add(tf.keras.layers.Dense(1,activation="sigmoid"))# then can switch the method. I will try too. if it imporves the prediction lets see. You also trymodel.summary())
        # Loss value for this batch.
        loss_value = loss_fn(y, logits)

    # Get gradients of loss wrt the weights.
    gradients = tape.gradient(loss_value, model.trainable_weights)

    # Update the weights of the model.
    optimizer_M.apply_gradients(zip(gradients, model.trainable_weights))

SyntaxError: ignored

In [ ]:
predictivemodel = Sequential()
predictivemodel.add(Dense(514, input_dim=514, W_regularizer=WeightRegularizer(l1=0.000001,l2=0.000001), init='normal'))
predictivemodel.add(Dense(257, W_regularizer=WeightRegularizer(l1=0.000001,l2=0.000001), init='normal'))
predictivemodel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])